In [ ]:
import os
import win32com.client
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

subject_filter = input("Введите название письма:").strip().lower()
TARGET_FOLDER_NAME = "Ошибки ПЦ/ВКА"

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item(1)

target_folder = None
for folder in root_folder.Folders:
    if folder.Name == TARGET_FOLDER_NAME:
        target_folder = folder
        break

if not target_folder:
    raise Exception(f"Папка '{TARGET_FOLDER_NAME}' не найдена")

# Ищем письмо
messages = target_folder.Items
messages.Sort("[ReceivedTime]", True)

for message in messages:
    if message.Class != 43:
        continue
    if subject_filter in message.Subject.lower():
        print(f"Найдено письмо: {message.Subject}")
        if message.Attachments.Count > 0:
            for i in range(1, message.Attachments.Count + 1):
                xlsx_attachment = message.Attachments.Item(i)
                if xlsx_attachment.FileName.endswith(".xlsx"):
                    temp_path = os.path.join(os.getenv("TEMP"), xlsx_attachment.FileName)
                    xlsx_attachment.SaveAsFile(temp_path)
                    
                    try:
                        errors_df = pd.read_excel(temp_path, engine='openpyxl')
                        print(f"✅ Успешно загружено {len(errors_df)} записей")
                    except Exception as e:
                        print(f"🚨 Ошибка чтения файла: {e}")
                    finally:
                        if os.path.exists(temp_path):
                            os.remove(temp_path)
                            print("Временный файл удалён")
                    break
        break
else:
    print("Письмо не найдено или не содержит вложений.")

filepath_inn = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\КА ИНН.xlsx"
filepath_mp = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\Приходы КА на ГМ(МП)_ фильтр месяц.xlsx"
filepath_mias = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\!Для новинок\МИАС.xlsx"
inn_df = pd.read_excel(filepath_inn, sheet_name="КА ИНН")
mp = pd.read_excel(filepath_mp)
mias = pd.read_excel(filepath_mias)

inn_df = inn_df.rename(columns={'Контрагент Код':'Код КА'})
inn_df['Контрагент ИНН'] = inn_df['Контрагент ИНН'].fillna('').astype(str)
inn_df = inn_df.dropna()

errors_df = errors_df.merge(inn_df[["Код КА", 'Контрагент ИНН']], on="Код КА", how='left')

errors_df['Контрагент ИНН'] = errors_df['Контрагент ИНН'].apply(lambda x: str(int(float(x))) if str(x).replace('.', '').isdigit() else str(x))
mp['Контрагент ИНН'] = mp['Контрагент ИНН'].apply(lambda x: str(int(float(x))) if str(x).replace('.', '').isdigit() else str(x))

mp['СЦ1'] = (mp['Контрагент ИНН'].fillna('').astype(str) + mp['РЦ по основному складу Наименование'].fillna('').astype(str) + mp['Лог категория ID'].fillna('').astype(str))
errors_df['СЦ1'] = (errors_df['Контрагент ИНН'].fillna('').astype(str) + errors_df['РЦ ОС'].fillna('').astype(str) + errors_df['Лог категория'].fillna('').astype(str))
mp = mp.drop_duplicates(subset='СЦ1')

errors_df = errors_df.merge(mp[['СЦ1','Контрагент Название']], on="СЦ1", how='left')
errors_df = errors_df.rename(columns={'Контрагент Название':'КА на МП'})

def clean_code(val):
    if isinstance(val, str) and val.isdigit():
        return int(val)
    return val
    
mias['Код КА'] = mias['Код КА'].apply(clean_code)
errors_df['Код КА'] = errors_df['Код КА'].apply(clean_code)

mias = mias.query("`Состояние ПЦ` == 'Активный'")
mias = mias[['Код КА','Лог. категория','Регион(округ)','РЦ', 'Тип поставки', 'Формат']]

RC = ['РЦ Тольятти (новый)', 'РЦ Ярославль', 'РЦ Оренбург Ленина', 'РЦ Кемерово', 'РЦ Дзержинск', 'РЦ Краснодар Индустриальный', 'РЦ Новосибирск Садовый (новый)', 
      'РЦ Зеленодольск', 'РЦ Ерзовка', 'РЦ Шахты', 'РЦ Тула', 'РЦ Сургут', 'РЦ Первоуральск', 'РЦ Стерлитамак', 'РЦ Колпино', 'РЦ Коломна', 'РЦ Славянск-на-Кубани', 
      'РЦ Батайск', 'РЦ Кропоткин', 'РЦ Тюмень', 'РЦ Пермь', 'РЦ Челябинск', 'РЦ Энгельс', 'РЦ Пенза', 'РЦ Киров', 'РЦ Дмитров', 'РЦ Астрахань Тинаки', 'РЦ Лермонтов', 
      'РЦ Великий Новгород', 'РЦ Мурманск', 'РЦ Ногинск (а)', 'РЦ Ижевск', 'РЦ Смоленск', 'РЦ Воронеж', 'РЦ Орел (Хардиково)', 'РЦ Новороссийск Цемдолина', 
      'РЦ Омск', 'РЦ Иваново', 'РЦ Тамбов', 'РЦ Шушары (а)']

mias_sc2 = mias.dropna()
mias_sc2['СЦ2'] = mias_sc2['Код КА'].fillna('').astype(str) + mias_sc2['РЦ'].fillna('').astype(str) + mias_sc2['Лог. категория'].fillna('').astype(str) + mias_sc2['Формат'].fillna('').astype(str)
mias_sc2 = mias_sc2.drop_duplicates(subset=['СЦ2'])

mias_sc3 = mias[mias['РЦ'].isna() | (mias['РЦ'] == 'NaN')]
mias_sc3['СЦ3'] = mias_sc3['Код КА'].fillna('').astype(str) + mias_sc3['Регион(округ)'].fillna('').astype(str) + mias_sc3['Лог. категория'].fillna('').astype(str) + mias_sc3['Формат'].fillna('').astype(str)
mias_sc3 = mias_sc3.drop_duplicates(subset=['СЦ3'])

errors_df['СЦ2'] = errors_df['Код КА'].fillna('').astype(str) + errors_df['РЦ ОС'].fillna('').astype(str) + errors_df['Лог категория'].fillna('').astype(str) + errors_df['Формат'].fillna('').astype(str)
errors_df['СЦ3'] = errors_df['Код КА'].fillna('').astype(str) + errors_df['Округ'].fillna('').astype(str) + errors_df['Лог категория'].fillna('').astype(str) + errors_df['Формат'].fillna('').astype(str)

errors_df = errors_df.merge(mias_sc2[['СЦ2','Код КА', 'Тип поставки']], on="СЦ2", how='left')
errors_df = errors_df.merge(mias_sc3[['СЦ3','Код КА', 'Тип поставки']], on="СЦ3", how='left')

errors_df = errors_df.rename(columns={'Тип поставки':'Тип поставки_1'})
errors_df = errors_df.rename(columns={'Тип поставки_y':'Тип поставки_2'})

errors_df = errors_df.rename(columns={'Код КА_y':'Код КА_1'})
errors_df = errors_df.rename(columns={'Код КА':'Код КА_2'})
errors_df = errors_df.rename(columns={'Код КА_x':'Код КА'})

#############################################################

errors_df['РЦ/МП'] = None

mask = errors_df['РЦ/МП'].isna()
errors_df.loc[mask, 'РЦ/МП'] = np.where(
    (errors_df.loc[mask, 'КА на МП'].notna()) &
    (errors_df.loc[mask, 'Формат'] == "БФ"), 
    'МП', 
    pd.NA
)

mask = errors_df['РЦ/МП'].isna()
errors_df.loc[mask, 'РЦ/МП'] = np.where(
    (errors_df.loc[mask, 'Код КА_1'].notna()) |
    (errors_df.loc[mask, 'Код КА_2'].notna()), 
    'РЦ ОС', 
    pd.NA
)

mask = errors_df['РЦ/МП'].isna()
errors_df.loc[mask, 'РЦ/МП'] = np.where(
    (errors_df.loc[mask, 'Код КА_1'].isna()) &
    (errors_df.loc[mask, 'Код КА_2'].isna()), 
    'Добавить в ПЦ (РЦ)', 
    pd.NA
)

errors_df['Тип поставки'] = np.where(
    errors_df['РЦ/МП'] == 'МП',
    'МП',
    errors_df['Тип поставки_1'].combine_first(errors_df['Тип поставки_2'])
)


# Выделяем строки с различиями
mismatched_df = errors_df[
    errors_df['Тип поставки-СУАГ:комплектация'] != errors_df['РЦ/МП']
]
mismatched_df['Тип поставки'] = mismatched_df['Тип поставки'].fillna('ПД')

if mismatched_df.empty:
    print("Все значения совпадают. Письмо отправлять не нужно.")
else:
    print(f"Обнаружено {len(mismatched_df)} несовпадений. Подготовка письма...")

    # # Сохраняем Excel с результатами
    # save_path = os.path.join(os.getenv("TEMP"), f"errors_updated_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
    # errors_df.to_excel(save_path, index=False)
    
    # Формируем текст письма
    mismatch_info = mismatched_df[['Код КА', 'Поставщик', 'РЦ ОС', 'Округ', 'Формат', 'Лог категория', 'Тип поставки']].drop_duplicates()
    mismatch_html = mismatch_info.to_html(index=False, border=1, justify='center')


    custom_body = f"""<html>
<body>
<p>Добрый день.</p>

{mismatch_html}

</body>
</html>"""

    # Отправка письма
    recipient_email = input("Введите email получателя: ").strip()
    forward = message.Forward()
    forward.To = recipient_email
    forward.BodyFormat = 2  # HTML
    forward.HTMLBody = custom_body
    # forward.Attachments.Add(save_path)

    try:
        forward.Send()
        print("Письмо отправлено успешно.")
    except:
        print("Ошибка отправки письма")